<a href="https://colab.research.google.com/github/jcauzi/Some-programs-I-made/blob/main/Semi%20supervized%20/%20auto-trained%20word-sense%20disambiguation%20model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1 : récupération des embeddings avec gensim

Pour l'instant, on les récupère sur ce lien : https://fauconnier.github.io/

Word2vec fair à partir du corpus frWac par Mr Jean-Philippe Fauconnier\n

Utilise un corpus de textes tirés du super journal français **Le Monde**

Taille des vecteurs d'embedding : 200



In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin", binary=True, unicode_errors="ignore")
model.save("word2vec.model")
#model.most_similar permet d'afficher les mots les plus similaires à un mot cible dans notre model w2v
#model.most_similar("voiture")

[('véhicule', 0.7415274381637573),
 ('voitures', 0.7131628394126892),
 ('automobiliste', 0.6627452373504639),
 ('garée', 0.6530455946922302),
 ('camionnette', 0.6363461017608643),
 ('portière', 0.6300318837165833),
 ('chauffeur', 0.6250183582305908),
 ('auto', 0.6175906658172607),
 ('bagnole', 0.6143242120742798),
 ('décapotable', 0.6119450330734253)]

# Partie 2 : parsing des données d'apprentissage (premier corpus fourni)

Parcour des données de train et constitution d'une liste d'exemples avec leur classe gold pour les 3 verbes.

In [ ]:

# ---- EXTRACTION DES DATAS SUPERVISEES POUR ABATTRE / AFFECTER / ABORDER -----#

from gensim.models import KeyedVectors
model = KeyedVectors.load("word2vec.model")

import pickle
import os
# from icecream import ic
from pprint import pprint

with open("affecter.deep_and_surf.sensetagged.conll","r") as file :
    with open("data","wb") as data :
        content = file.read()
        text = []
        flex_list =[]
        for phrase in content.split("\n\n") :
            if len(phrase) > 0:
                temp = ""
                sens = ""
                for line in phrase.split("\n") :
                    tmp = line.split("\t")
                    if tmp[2] == "affecter" :
                        #J'AI RAJOUTE CETTE CONDITION (car certaines formes fléchies
                        #n'ont pas d'embedding donc on ne peut pas intégrer le phrase dans le corpus.)
                        if not exists(tmp[1]) :
                          break
                        #split sur | pour sur # puis on récupère le numéro du sens
                        sens = [i for i in tmp[5].split("|") if "sense" in i][0].split("#")[1]
                        #récupérer la forme fléchie
                        if tmp[1] not in flex_list:
                          flex_list.append(tmp[1])
                    # checker si existe dans embedding
                    if exists(tmp[1]):
                      temp += tmp[1] + " "
            # récupère suelement si le sens a un numéro
            if len(sens) == 1:
              text.append((temp,sens))
        pickle.dump(text, data)


# POUR DÉSÉRIALIZER
with open ('data', 'rb') as data:
  train_corpus_sent = pickle.load(data)

print({sens[1] for sens in train_corpus_sent})

#petite fonction pour regarder les phrases d'un certain sens :
for example in train_corpus_sent :
  if example[1] == "3" :
    print(example[0])

# print(flex_list)


#sens 1 : abattre des batiments / arbres / avions / 
#sens 2 : abattre des humains / tuer
#sens 3 : décourager 
#sens "" : geste d'abattre qqchose (carte, épée)


set()


Extraction des exemples non annotés du corpus Est Républicain


In [ ]:
# ---------------- EXTRACTION DES DATAS NON-SUPERVISEES -----------------------#

#extraction des phrases pour corpus non supervisé.
# ---------------------- EXTRACTION DES DATAS POUR ABATTRE ---------------------- #
import pickle
import os
# from icecream import ic
import gensim
from gensim.models import KeyedVectors


list = ['aboutir', 'investir', 'traduire', 'témoigner', 'juger', 'justifier',
        'viser', 'prononcer', 'accomplir', 'convenir', 'acquérir', 'achever',
        'observer', 'adapter', 'admettre', 'entraîner', 'payer', 'respecter',
        'affecter', 'demeurer', 'aggraver', 'agir', 'ajouter', 'alimenter',
        'coûter', 'relancer', 'préférer', 'appliquer', 'apporter', 'fonder',
        'appuyer', 'changer', 'chuter', 'soutenir', 'concevoir', 'interroger',
        'confirmer', 'transformer', 'manifester', 'interpeller', 'signer',
        'rester', 'tuer', 'indiquer', 'conduire', 'situer', 'aider',
        'poursuivre', 'profiter', 'détenir', 'lire', 'contenir', 'dominer',
        'noter', 'dater', 'adopter', 'enregistrer', 'intervenir', 'conclure',
        'disputer', 'estimer', 'appartenir', 'arriver', 'chercher', 'composer',
        'confier']

#model = KeyedVectors.load("word2vec.model")

def exists(word) :
    # in : a word string
	  # out:
	  # checks whether a given word has a corresponding embedding in our w2v model
    try:
        embedding_vector = model[word]
        return True
    except: return False


def extract(filename, lemma) :
    #extracts the sentences with occurences of the lemma from a single file of the corpus Est Républicain
    #in : the name of the file containing the sentences / the lemma in str
    #out : the list of sentences containing occurences of the lemma 
    #footnote :: a single sentence is in the format [word_list,{"flex_form":str,"lemma":str}]
    text = []
    flex_list =[]
    with open(filename, encoding="utf8", errors='ignore') as file :
        with open("data","wb") as data :
            content = file.read()
            for phrase in content.split("\n\n") :
                contains = False
                if len(phrase) > 0:
                    for line in phrase.split("\n") :
                        word = line.split("\t")
                        if len(word) > 3 :
                            if word[2] == lemma :
                                contains = True
                                flex = word[1]
                                if flex not in flex_list:
                                    flex_list.append(flex)
                if contains :
                    sentence = ""
                    for line in phrase.split("\n") :
                        if exists(line.split("\t")[1]):
                            sentence += line.split("\t")[1] + " "
                    word_list = sentence.split(" ")
                    text.append([word_list, [{"flex_form" : flex, "lemma" : lemma}]])
            return text


def extractall_one(dirname, lemma) :
    #extracts all sentences containing an occurence of the lemma in a set of files from the corpus Est Républicain
    #in : the name of the directory containing the set of files
    #out : the list of sentences containing occurences of the lemma 
    text = []
    i = 0
    for filename in os.listdir(dirname):
        i+=1
        new = extract(dirname + "/" + filename, lemma)
        text = text + new
        print(f"extraction occurences '{lemma}' fichier {i}/27")
    return text


#print(extractall_one("estrepublicain", "affecter")[0])

def extractall_all(dirname, lemma_list) :
    #extracts all sentences containing an occurence of one of the lemmas in a lemma list
    #in : the name of the directory containing the set of files from the corpus
    #out : the liste of sentences containing occurences of one of the lemmas
    full_list = []
    i = 0
    for lemma in lemma_list :
        i+=1
        full_list.extend(extractall_one(dirname, lemma))
        print(f"fin de l'extraction occurences '{lemma}' : {i}/{len(lemma_list)}")
    return full_list

# test = extractall_all("estrepublicain", list[0 : 3])

#serialization functions :
file_name = "all#noclass.pkl"
# open_file = open(file_name, "wb")
# pickle.dump(extractall_all("estrepublicain", list), open_file)
# open_file.close()



open_file = open(file_name, "rb")
list = pickle.load(open_file)
open_file.close()
print(list[0])
print(list[1000])
print(list[10000])
print(len(list))


incident qui n' affecte pas le véritable amateur comme le rappelle le commissaire-priseur « ont bourré leurs d' électronique et c' est toujours avec une légère suspicion qu' on pénètre dans une telle voiture 


Extraction des données annotées du corpus FSE entier

In [ ]:
#!/usr/bin/env python
import re
from bs4 import BeautifulSoup as Soup
import pickle
import io

def extract_data(data_filename,annotations_filename):

	#collecting gold annotations
	with open(annotations_filename) as file:
		lines = file.readlines()
	
	# dict list to garner relevant information
	word_dicts = {}

	for line in lines:
		all_data = line.split('__')
		token_id = all_data[0][:-1]
		val_inf = all_data[1].split('_')
		word_dicts[token_id] = { "lemma" : val_inf[2], "word sense" : val_inf[1], "sentence_id" : token_id[1:]}

	# word_dicts format : {token_id : {"lemma":...,"word sense" : ...,"sentence_id";...}}

	# xml parsing of the FSE file
	soup = Soup(open(data_filename),features="lxml")

	# list of sentences in the following format for a single sentence : [word_list,{}]
	sentences = []
	for sentence in soup.findAll("sentence"):
		instances = sentence.findAll("instance")
		words = sentence.text
		sentences.append([words.split(),[{"flex_form" : instance.text, "id" : str(instance['id']),"lemma" : word_dicts[instance['id']].get('lemma'),"word sense" : word_dicts[instance['id']].get('word sense')} for instance in instances]])

	# in order to partition the data by lemma, storing data in the the wsd_data dict
	wsd_data = {}
	for sentence in sentences:
		for item in sentence[1]:
			wsd_data[item["lemma"]] = wsd_data.get(item["lemma"],[])+[sentence]
    
	return wsd_data

# uncomment to run the extraction
#fse_data = extract_data("/Users/leolabat/Downloads/FSE-1.1-191210/FSE-1.1.data.xml","/Users/leolabat/Downloads/FSE-1.1-191210/FSE-1.1.gold.key.txt")

# uncomment to retrieve the pickled version of the dictionarized FSE data
# with open('/Users/leolabat/Desktop/WSD_project/fse_data.pickle','rb') as handle:
# 	fse_data = pickle.load(handle)

# Partie 3 : classifieur supervisé

Permet de créer des vecteurs de contextes comme addition des embeddings des mots dans un contexte autour du mot cible (forme fléchie du mot cible comprise). 

- deux classifieurs permettant de prédire le sens du mot cible dans un contexte non-vu à l'apprentissage (SVM et MLP) 
- une partition du corpus en train / test pour évaluer les performances du classifieur.
- des création de tableaux pour stocker les résultats.

In [ ]:

import pickle
import gzip
import gensim
import logging
import numpy as np
from gensim.models import KeyedVectors
from sklearn import svm
from sklearn import metrics
import random
import statistics
from collections import Counter
import pandas as pd
from statistics import pstdev

# collecting the dictionary with gold annotations from FSE data
with open('fse_data.pickle','rb') as handle:
    fse_data = pickle.load(handle)

# collecting embedding model
model = KeyedVectors.load("word2vec.model")

#collecting new data from the sskmeans :
file_name = "newdata.pkl"
open_file = open(file_name, "rb")
newdata = pickle.load(open_file)
open_file.close()

# custom-made functions
def cosine_similarity(vec1, vec2) :
	# in : two vectors of similar shape
    # out : cosine similarity between the two vectors
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def exists(word) :
	# in : a word string
	# out:
	# checks whether a given word has a corresponding embedding in our w2v model
    try:
        embedding_vector = model[word]
        return True
    except: return False

def train_corpus(sentences,lemma, window=4):
	# in : a list of sentences and a string for the lemma
	# footnote :: a single sentence is in the format [word_list,{"flex_form":str,"lemma":str}]
	# out : a list of pairs : (context_vector,gold_class)

	vectorized_train = []

	for sentence in sentences:
		for flex_form in sentence[1]:
			if flex_form.get("lemma") == lemma:
				vectorized_train.append((get_context(sentence[0], flex_form.get('flex_form'), window), flex_form.get('word sense')))
	return vectorized_train

def vectorize(sentences,lemma):
	# in : a list of sentences (format : [word_list,{"flex_form":str,"lemma":str}]) and a str for the lemma
	# out : a list of vectors corresponding to the contextualised vector representations of lemma occurrences

    vectorized = []
    for sentence in sentences:
        for flex_form in sentence[1]:
            if flex_form.get("lemma") == lemma:
                try:
                    vectorized.append(get_context(sentence[0],flex_form.get('flex_form')))
                except: continue
    return vectorized



def get_context(word_list,flex_form,window=4):
	# in : word list corresponding to the sentence the target word was taken from, the occurrence itself (flex_form), the context window size
	# out : a single vector corresponding to the in-place addition of fixed embeddings, thus encapsulating the target word

	# checking if there is an issue with the flex_form
    if flex_form in word_list:
        index = word_list.index(flex_form)
    else:
        print("flex_form not in word list")
        raise ValueError("flex_form not in word list")

    context = []
    for i in range(1, window+1):
        if not index-i < 0 :
            #if the word is unavailable in our embeddings it won't be taken into account (=vectorized into a null vector)
            if exists(word_list[index-i]) : context.append(model[word_list[index-i]])
        if index+i < len(word_list) :
            if exists(word_list[index+i]) : context.append(model[word_list[index+i]])
    # if the flex_form exists, it is added as well (obviously)
    if exists(flex_form):
    	context.append(model[word_list[index]])

    #first context word as initialization
    context_embed = context[0]
    # in-place addition of all the other vectors
    for i in range(1, len(context)) :
        context_embed = np.add(context_embed, context[i])
    return context_embed

def baseline(corpus) :
	# in : annotated corpus in its vectorized form(!)
	# out : a list of tuples (word sense,word sense frequency)

	Y = []
	for example in corpus :
		Y.append(example[1])
		c = Counter(Y)
	#print("Most frequent baseline : ", round(max([(c[i] / len(Y) * 100.0) for i in c]), 2), "%")
	return round(max([(c[i] / len(Y) * 100.0) for i in c]), 2)


##
##
## PLAIN CLASSIFICATION
##
##

def SVM_WSD(data,lemma,partition,n_trials,nb_iter_svm):
    # in :
    #      data : lemma dictionary with list of annotated data as values
    #      lemma : string corresponding to the lemma undergoing disambiguation
    #      partition : float of value between 0 and 1 (excluded) to partition the data into train/test
    #      n_trials : number of trials
    #      nb_iter_svm : max_iter parameter for the svm
    # out : average accuracy of the SVM algorithm on a fraction of the annotated data

    # to avoid overwriting our data (precautionary measure)
    examples = data.get(lemma).copy()

    # used to partition the data into a train_set and a test_set
    size = len(examples)
    tr = int(size*partition)

    # vectorization of the training set
    training_set = train_corpus(examples,lemma)

    print(f'Taille des données annotées : {size} exemples')

    baseline(training_set)

    accuracy_list = []
    for i in range(n_trials) :
        # randomize the parition of the annotated data
        random.shuffle(training_set)

        X, Y = [], []
        for example in training_set[0:tr] :
            X.append(example[0])
            Y.append(example[1])

        if len(Y)<=1 :
            break

        X_test, Y_gold = [], []
        for example in training_set[tr:] :
            X_test.append(example[0])
            Y_gold.append(example[1])

        # Uncomment to add the new data from the SSK-mean as train examples
        # for i in range(len(newdata[lemma])) :
        #     X.append(newdata[lemma][i][0])
        #     Y.append(newdata[lemma][i][1])


      #max iter un peu plus grand que 1000 pour ne pas avoir de warning de non-convergence
        lin_clf = svm.LinearSVC(max_iter=nb_iter_svm)
        lin_clf.fit(X, Y)
        Y_pred = lin_clf.predict(X_test)

        accuracy_list.append(metrics.accuracy_score(Y_gold, Y_pred))
    return accuracy_list
    #print(f"SVM accuracy for \"{lemma}\" : {round(statistics.mean(accuracy_list)*100, 2)}%")



def data(lemma, window):
    #collects data for various hyperparameters : lemma, context window
    data = []
    for num in window:
        train = train_corpus(fse_data.get(lemma).copy(), lemma, num)
        #if len(data) == 0:
        acc = SVM_WSD(fse_data, lemma, 0.8, 10, 10000)
        data.append([f"{lemma} / {baseline(train)}", num,
        f"{round(max(acc)*100, 2)}%",
        f"{round(min(acc)*100, 2)}%",
        f"{round(statistics.mean(acc)*100, 2)}%",
        round(pstdev(acc), 4)])

    return data

# table creation :
columns = ["lemma / baseline", "window", "max", "min", "mean", "standard deviation"]
window = [4]
# list of the 18 words used for semi-supervised auto-trained classification
list = ['traduire', 'viser', 'prononcer', 'convenir', 'achever', 'adapter',
        'alimenter', 'coûter', 'relancer', 'appuyer', 'chuter', 'confirmer',
        'manifester', 'conduire', 'aider', 'détenir', 'contenir', 'dater', 'confier']

tableau = pd.DataFrame(data=data(list[0], window), columns = columns)
for lemma in list[1:] :
    df1 = pd.DataFrame(data=data(lemma, window) , columns = columns)
    tableau = pd.concat([tableau,df1], ignore_index=True)
    print(tableau)
tableau.to_csv("tables/supervized_SVM19.csv")



Classifieur par réseau de neurones à couche cachée (Multi-Layered Perceptron)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_accuracy = []

for i in range(100) :

  random.shuffle(train_corpus)

  X, Y = [], []
  for example in train_corpus[0:140] :
    X.append(example[0])
    Y.append(example[1])

  X_test, Y_gold = [], []
  for example in train_corpus[140:165] :
    X_test.append(example[0])
    Y_gold.append(example[1])
  #max iter un peu plus grand que 1000 pour ne pas avoir de warning de non-convergence
  mlp = MLPClassifier(hidden_layer_sizes=100,activation='tanh',solver='sgd',random_state=1,max_iter=5000)
  mlp.fit(X, Y)
  Y_pred = mlp.predict(X_test)

  mlp_accuracy.append(metrics.accuracy_score(Y_gold, Y_pred))

print(f"mlp accuracy : {statistics.mean(mlp_accuracy)}")

Partie 5 : clustering semi-supervisé 

SSK-mean pour l'extraction de nouveaux exemples de train

In [ ]:
import numpy as np
import pickle
import random
from statistics import pstdev
from wsd_plain_classification import *

with open('fse_data.pickle','rb') as handle:
    fse_data = pickle.load(handle)

def get_key(my_dict,val):
    for key, value in my_dict.items():
        if val == value:
            return key
    return "key doesn't exist"

class SSK_Means():
    def __init__(self, k=2, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

    def fit(self,data,labeled_data,labels):

    	# initialisation d'un dictionnaire de centroïdes
        self.centroids = {}

        #récupération des données étiquetées
        self.labeled_data = labeled_data
        # récupération des étiquettes gold
        self.gold_labels = set(labels)
        # agrégation des clusters connus
        self.clusters = {}
        for label in self.gold_labels:
        	self.clusters[label] = []

        # attribution des données étiquettées aux clusters gold
        for featset,label in zip(labeled_data,labels):
        	self.clusters[label].append(featset)

        # calcul des centroïdes à partir des premières données
        for label in self.gold_labels:
            self.centroids[label] = np.average(self.clusters[label],axis=0)

        # pour chaque epoch
        for i in range(self.max_iter):

        	# initialisation d'un dictionnaire pour les appariements aux clusters
            self.classifications = {}
            #print(self.gold_labels)
            # initialisation de listes comme valeurs pour k clés
            for label in self.gold_labels:
                #print(f'label : {label}')
                self.classifications[label] = []

            for featureset in data:
                distances = {centroid : np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids}
                classification = get_key(distances, min(distances.values()))
                self.classifications[classification].append(featureset)

            for featset,label in zip(labeled_data,labels):
        	    self.classifications[label].append(featset)

            # sauvegarde des barycentres ayant servi à la classification de l'epoch
            prev_centroids = dict(self.centroids)

            # recalcul des nouveaux barycentres à partir de la moyenne des valeurs qu'ils partitionnent
            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    #print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        distances = {centroid : np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids}
        classification = get_key(distances,min(distances.values()))
        return classification

hc_path = "all#noclass.pkl"

with open(hc_path,'rb') as handle:
    huge_corpus = pickle.load(handle)

# turn the unclassified data into a lemma-based dictionary
lemma_data = {}
for sentence in huge_corpus:
    for item in sentence[1]:
        lemma_data[item["lemma"]] = lemma_data.get(item["lemma"],[])+[sentence]

def SSKMeans_classifier(known_data,unknown_data,lemma,n_trials,partition):

    # récupération des données connues pour le classifieur
    known = known_data[lemma].copy()
    train = train_corpus(known,lemma)
    word_sense_nb = len(set([lab for item,lab in train]))

    #vectorizing unknown data
    data = vectorize(unknown_data[lemma],lemma)


    #pas de partition des données en train et test car test inutile
    random.shuffle(train)
    training_set = train
    labeled_data = []
    labels = []
    for example,label in training_set:
        labeled_data.append(example)
        labels.append(label)

    Kmeans = SSK_Means()
    Kmeans.fit(data,labeled_data,labels)
    predicted = []
    for example in data:
        pred = Kmeans.predict(example)
        predicted.append((example, pred))
    return predicted

#constitution d'un disctionnaire pour récolter les nouveaux exemples 
lemmadic = {}
lemmalist = ['traduire', 'viser', 'prononcer', 'convenir', 'achever', 'adapter', 'alimenter', 'coûter', 'relancer', 'appuyer', 'chuter', 'confirmer', 'manifester', 'conduire', 'aider', 'détenir', 'contenir', 'dater', 'confier']
for lemma in lemmalist :
    lemmadic[lemma] = SSKMeans_classifier(fse_data,lemma_data,lemma,100,0.8)

#sérialisation
file_name = "newdata.pkl"
open_file = open(file_name, "wb")
pickle.dump(lemmadic, open_file)
open_file.close()
